In [1]:
import clr
clr.AddReference('C:\\Program Files\\Siemens\\Automation\\Portal V16\PublicAPI\\V16\\Siemens.Engineering.dll')
from System.IO import DirectoryInfo, FileInfo
import Siemens.Engineering as tia
import Siemens.Engineering.HW.Features as hwf
import Siemens.Engineering.Compiler as comp
import Siemens.Engineering.Download as dl
import os



# Starting TIA and creating project


In [2]:
#Starting TIA with UI, also possible to start without ui
print ('Starting TIA with UI')
mytia = tia.TiaPortal(tia.TiaPortalMode.WithUserInterface)


Starting TIA with UI


In [3]:
# Alternative code to connect to an allready running instance (uncomment to use)

processes = tia.TiaPortal.GetProcesses() # Making a list of all running processes
print (processes)
#process = processes[0]                   # Just taking the first process as an example
#mytia = process.Attach()
#myproject = mytia.Projects[0]


In [4]:
# Creating a new project. Using try/except in case project allready exists

project_path = DirectoryInfo ('C:\\Jonas\\TIA')
project_name = 'PythonTest'
try:
    myproject = mytia.Projects.Create(project_path, project_name)
except Exception as e:
    print (e)

# Adding HW to the project

In [5]:
#Adding the main components

print ('Creating PLC1')
PLC1_mlfb = 'OrderNumber:6ES7 513-1AL02-0AB0/V2.6'
PLC1 = myproject.Devices.CreateWithItem(PLC1_mlfb, 'PLC1', 'PLC1')


print ('Creating IOnode1')
IOnode1_mlfb = 'OrderNumber:6ES7 155-6AU01-0BN0/V4.1'
IOnode1 = myproject.Devices.CreateWithItem(IOnode1_mlfb, 'IOnode1', 'IOnode1')


print ('Creating HMI1')
HMI1_mlfb = 'OrderNumber:6AV2 124-0GC01-0AX0/15.1.0.0'
HMI1 = myproject.Devices.CreateWithItem(HMI1_mlfb, 'HM1', None)

#ToDo Add start screen to avoid compilation error fo the HMI

Creating PLC1
Creating IOnode1
Creating HMI1


In [6]:
# Adding IO cards to the PLC and IO station
# This is basic to show how it works, use loops with checks (CanPlugNew) to see if the slot is available
# CanPlugnew is not mandatory, but usefull in real code

if (PLC1.DeviceItems[0].CanPlugNew('OrderNumber:6ES7 521-1BL00-0AB0/V2.1','IO1',2)): 
    PLC1.DeviceItems[0].PlugNew('OrderNumber:6ES7 521-1BL00-0AB0/V2.1','IO1', 2)

    
if (IOnode1.DeviceItems[0].CanPlugNew('OrderNumber:6ES7 131-6BH01-0BA0/V0.0','IO1',1)):
    IOnode1.DeviceItems[0].PlugNew('OrderNumber:6ES7 131-6BH01-0BA0/V0.0','IO1', 1)




# Creating network, iosytem and setting IP adresses


In [7]:
#creating a list of all found network interfaces on all stations in the station list
n_interfaces = []
for device in myproject.Devices:
    device_item_aggregation = device.DeviceItems[1].DeviceItems
    for deviceitem in device_item_aggregation:
        network_service = tia.IEngineeringServiceProvider(deviceitem).GetService[hwf.NetworkInterface]()
        if type(network_service) is hwf.NetworkInterface:
            n_interfaces.append(network_service)


In [8]:
# Assigning an IP to each item in the list (dirty code, but to demonstrate how theAPI works)

n_interfaces[0].Nodes[0].SetAttribute('Address','192.168.0.130')
n_interfaces[1].Nodes[0].SetAttribute('Address','192.168.0.131')
n_interfaces[2].Nodes[0].SetAttribute('Address','192.168.0.132')

In [9]:
# Creating subnet and IO system on the first item in the list
# Connects to subnet for remaining devices, if IO device it gets assigned to the IO system
for n in n_interfaces:
    if n_interfaces.index(n) == 0:
        subnet = n_interfaces[0].Nodes[0].CreateAndConnectToSubnet("Profinet")
        ioSystem = n_interfaces[0].IoControllers[0].CreateIoSystem("PNIO");
    else:
        n_interfaces[n_interfaces.index(n)].Nodes[0].ConnectToSubnet(subnet)
        if (n_interfaces[n_interfaces.index(n)].IoConnectors.Count) >> 0:
            n_interfaces[n_interfaces.index(n)].IoConnectors[0].ConnectToIoSystem(ioSystem);

# Compiling HW & SW


In [10]:
# Defining method to recursively print error messages
def print_comp(messages):
    for msg in messages:
        print(f'Path: {msg.Path}')
        print(f'DateTime: {msg.DateTime}')
        print(f'State: {msg.State}')
        print(f'Description: {msg.Description}')
        print(f'Warning Count: {msg.WarningCount}')
        print(f'Error Count: {msg.ErrorCount}\n')
        print_comp(msg.Messages)
    
# Compiling all devices
for device in myproject.Devices:
    compile_service =  device.GetService[comp.ICompilable]()
    result = compile_service.Compile()
                
    #Printing results from compiler
    print(f'State: {result.State}')
    print(f'Warning Count: {result.WarningCount}')
    print(f'Error Count: {result.ErrorCount}')
    print_comp(result.Messages)   

        

State: 2
Warning Count: 2
Error Count: 0
Path: PLC1
DateTime: 09/03/2020 17:48:10
State: 2
Description: 
Warning Count: 2
Error Count: 0

Path: Hardware configuration
DateTime: 09/03/2020 17:48:12
State: 2
Description: 
Warning Count: 2
Error Count: 0

Path: PLC1
DateTime: 09/03/2020 17:48:19
State: 2
Description:  
Warning Count: 2
Error Count: 0

Path: Rail_0
DateTime: 09/03/2020 17:48:19
State: 2
Description:  
Warning Count: 2
Error Count: 0

Path: PLC1
DateTime: 09/03/2020 17:48:19
State: 2
Description:  
Warning Count: 2
Error Count: 0

Path: PLC1
DateTime: 09/03/2020 17:48:19
State: 2
Description:  
Warning Count: 1
Error Count: 0

Path: 
DateTime: 09/03/2020 17:48:19
State: 2
Description: PLC1 does not contain a configured protection level
Warning Count: 1
Error Count: 0

Path: CPU display_1
DateTime: 09/03/2020 17:48:19
State: 2
Description:  
Warning Count: 1
Error Count: 0

Path: 
DateTime: 09/03/2020 17:48:19
State: 2
Description: The S7-1500 CPU display does not contain an

# Option to compile SW only


In [11]:
# Defining method to recursively print error messages
def print_comp(messages):
    for msg in messages:
        print(f'Path: {msg.Path}')
        print(f'DateTime: {msg.DateTime}')
        print(f'State: {msg.State}')
        print(f'Description: {msg.Description}')
        print(f'Warning Count: {msg.WarningCount}')
        print(f'Error Count: {msg.ErrorCount}\n')
        print_comp(msg.Messages)
    
#compiling all sw in all devices
for device in myproject.Devices:
    device_item_aggregation = device.DeviceItems
    for deviceitem in device_item_aggregation:   
            software_container = tia.IEngineeringServiceProvider(deviceitem).GetService[hwf.SoftwareContainer]()
            if (software_container != None):
                print(f'compiling: {deviceitem.Name}')
                software_base = software_container.Software
                
                compile_service =  software_base.GetService[comp.ICompilable]()
                result = compile_service.Compile()
                
                #Printing results from compiler
                print(f'State: {result.State}')
                print(f'Warning Count: {result.WarningCount}')
                print(f'Error Count: {result.ErrorCount}')
                print_comp(result.Messages)   


compiling: PLC1
State: 0
Warning Count: 0
Error Count: 0
Path: PLC1
DateTime: 09/03/2020 17:49:44
State: 1
Description: 
Warning Count: 0
Error Count: 0

Path: Program blocks
DateTime: 09/03/2020 17:49:45
State: 1
Description: 
Warning Count: 0
Error Count: 0

Path: None
DateTime: 09/03/2020 17:49:45
State: 1
Description: No block was compiled. All blocks are up-to-date.
Warning Count: 0
Error Count: 0

Path: None
DateTime: 09/03/2020 17:49:47
State: 0
Description: Compiling finished (errors: 0; warnings: 0)
Warning Count: 0
Error Count: 0

compiling: HMI_RT_1
State: 3
Warning Count: 0
Error Count: 1
Path: HM1
DateTime: 09/03/2020 17:49:49
State: 3
Description: 
Warning Count: 0
Error Count: 1

Path: None
DateTime: 09/03/2020 17:49:49
State: 1
Description: Software compilation started.
Warning Count: 0
Error Count: 0

Path: None
DateTime: 09/03/2020 17:49:49
State: 1
Description: No HMI runtime-relevant changes have been made since the last error-free HMI compilation. New compilation i

# Exporting 

    

In [12]:
#Optional code to remove xml files that may allready exist on your computer
try:
    os.remove('C:\\Jonas\\TIA\\exports\\dummy.xml')
except OSError:
    pass
try:
    os.remove('C:\\Jonas\\TIA\\exports\\Main.xml')
except OSError:
    pass



In [13]:
# exporting "main" from PLC1

#export_path = FileInfo ('C:\\Jonas\\TIA\\exports\\Main.xml')
software_container = tia.IEngineeringServiceProvider(PLC1.DeviceItems[1]).GetService[hwf.SoftwareContainer]()
software_base = software_container.Software
plc_block = software_base.BlockGroup.Blocks.Find("Main")
plc_block.Export(FileInfo('C:\\Jonas\\TIA\\exports\\Main.xml'), tia.ExportOptions.WithDefaults)

# Exporting tagtable from PLC1
tag_table_group = software_base.TagTableGroup
#creating a dummy table to export
tagtable = tag_table_group.TagTables.Create("dummy")
tagtable = tag_table_group.TagTables.Find("dummy")
tagtable.Export(FileInfo('C:\\Jonas\\TIA\\exports\\dummy.xml'), tia.ExportOptions.WithDefaults)


#deleting block and tag table in project 
plc_block.Delete()
tagtable.Delete()


# Importing


In [14]:
# Importing the xml files back in to the project
tag_table_group.TagTables.Import(FileInfo('C:\\Jonas\\TIA\\exports\\dummy.xml'), tia.ImportOptions.Override)
software_base.BlockGroup.Blocks.Import(FileInfo('C:\\Jonas\\TIA\\exports\\Main.xml'), tia.ImportOptions.Override)


In [18]:
myproject.Save()

AttributeError: 'Project' object has no attribute 'Saveee'

In [16]:
myproject.Close()

In [17]:
mytia.Dispose()

In [16]:
download_provider = tia.IEngineeringServiceProvider(PLC1.DeviceItems[1]).GetService[dl.DownloadProvider]()
configuration = download_provider.Configuration
for i in configuration.Modes:
    print (i.Name)


PN/IE


In [17]:
#configuration_mode = configuration.Modes.Find("PN/IE")
# accessing directly first element, and printing to verify
configuration_mode = configuration.Modes[0] 
print(configuration_mode.Name)


PN/IE


In [18]:
for i in configuration_mode.PcInterfaces:
    print(i.Name)

PLCSIM


In [19]:
target_configuration = configuration_mode.PcInterfaces[0].TargetInterfaces[0]
print(target_configuration.Name)

1 X1


In [20]:
def PreConfigureDownload(downloadConfiguration):
        print("en")
        stopModules = dl.Configurations.StopModules(downloadConfiguration)
        if (stopModules != None):
            stopModules.CurrentSelection = dl.Configurations.StopModulesSelections.StopAll
            return
        print("to")

        alarmTextLibraries = dl.Configurations.AlarmTextLibrariesDownload(downloadConfiguration)
        if (alarmTextLibraries != None):
            alarmTextLibraries.CurrentSelection = dl.Configurations.AlarmTextLibrariesDownloadSelections.ConsistentDownload
            return
        
        blockBindingPassword =  dl.Configurations.BlockBindingPassword(downloadConfiguration)
        if (blockBindingPassword != None):
            password = None
            dl.Configurations.blockBindingPassword.SetPassword(password)
            return
                   
        checkBeforeDownload = dl.Configurations.CheckBeforeDownload(downloadConfiguration)
        if (checkBeforeDownload != None):
            checkBeforeDownload.Checked = true
            return
        
        consistentBlocksDownload = dl.Configurations.ConsistentBlocksDownload(downloadConfiguration)
        if (consistentBlocksDownload != None):
            consistentBlocksDownload.CurrentSelection = dl.Configurations.ConsistentBlocksDownloadSelections.ConsistentDownload
            return 
            
        moduleWriteAccessPassword = dl.Configurations.ModuleWriteAccessPassword(downloadConfiguration)
        if (moduleWriteAccessPassword != None):
            password = None; 
            moduleWriteAccessPassword.SetPassword(password)
            return 
            
       


In [21]:
def PostConfigureDownload(downloadConfiguration):
    startModules = dl.Configurations.StartModules(downloadConfiguration)
    if (startModules != None):
        startModules.CurrentSelection = dl.Configurations.StartModulesSelections.StartModule
    

In [22]:

pre = dl.DownloadConfigurationDelegate(PreConfigureDownload)
post = dl.DownloadConfigurationDelegate(PostConfigureDownload)
result = download_provider.Download(target_configuration,pre, post, dl.DownloadOptions.Hardware | dl.DownloadOptions.Software )

NonRecoverableException: Error when calling method 'Download' of type 'Siemens.Engineering.Download.DownloadProvider'.
   at Siemens.Engineering.Private.Session.Siemens.Engineering.Private.IInstanceSession.InvokeFunc[T,TC](LifetimeContractHandle`1 lifetimeContractHandle, String name, IEnumerable`1 parameters, String fullName)
   at Siemens.Engineering.Private.InternalInstanceAccess`2.InvokeFunc[T](String name, IEnumerable`1 parameters)
   at Siemens.Engineering.Download.DownloadProvider.Download(IConfiguration configuration, DownloadConfigurationDelegate preDownloadConfigurationDelegate, DownloadConfigurationDelegate postDownloadConfigurationDelegate, DownloadOptions downloadOptions)

In [73]:
print(dir(download_provider.Download))

['__call__', '__class__', '__delattr__', '__delete__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__set__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [114]:
print(type(dl.DownloadConfigurationDelegate(post)))

<class 'Siemens.Engineering.Download.DownloadConfigurationDelegate'>


In [105]:
import System

Action = getattr(System, "Action`1")
Action2 = getattr(System, "Action`2")
preDownloadDelegate = Action[dl.Configurations.DownloadConfiguration](PreConfigureDownload)
postDownloadDelegate = Action2[dl.Configurations.DownloadConfiguration](PostConfigureDownload)
result = download_provider.Download(target_configuration, 
                                    preDownloadDelegate,
                                    postDownloadDelegate, 
                                    dl.DownloadOptions.Hardware 
                                    dl.DownloadOptions.Software
                                   )

TypeError: No method matches given arguments for Download

nn
